In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pymongo

In [2]:
#Imports & Dependencies
!pip install selenium
!pip install splinter
from splinter import Browser
from bs4 import BeautifulSoup

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
# Set up chromedrive
executable_path = {"executable_path": "/Users/gblac/Desktop/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

# Use BeautifulSoup to scrape website for article info

In [4]:
# Create Soup
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [5]:
# retrieve article title and text
article = soup.find("div",class_="list_text")
articletext = soup.find("div","article_teaser_body").text
articletitle = soup.find("div",class_="content_title").text
print(articletitle)
print(articletext)
print(article)

NASA Launches a New Podcast to Mars
NASA's new eight-episode series 'On a Mission' follows the InSight spacecraft on its journey to Mars and details the extraordinary challenges of landing on the Red Planet.
<div class="list_text"><div class="list_date">October 29, 2018</div><div class="content_title"><a href="/news/8381/nasa-launches-a-new-podcast-to-mars/" target="_self">NASA Launches a New Podcast to Mars</a></div><div class="article_teaser_body">NASA's new eight-episode series 'On a Mission' follows the InSight spacecraft on its journey to Mars and details the extraordinary challenges of landing on the Red Planet.</div></div>


In [6]:
# Create Soup
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [7]:
# Find featured image URL
img = soup.find("img",class_="thumb")['src']
featured_image_url = image_url + img
print(img)
print(featured_image_url)

/spaceimages/images/wallpaper/PIA22793-640x350.jpg
https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA22793-640x350.jpg


# Use Pandas pd.read_html to read an html table

In [8]:
import pandas as pd
facts_url = "https://space-facts.com/mars/"
mars_data = pd.read_html(facts_url)
mars_data = pd.DataFrame(mars_data[0])
mars_facts = mars_data.to_html(header = False, index = False)
print(mars_facts)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Use Tweepy to get Mars Weather 

In [9]:
# API.user_timeline([id/user_id/screen_name][, since_id][, max_id][, count][, page])
# id – Specifies the ID or screen name of the user.
# user_id – Specifies the ID of the user. Helpful for disambiguating when a valid user ID is also a valid screen name.
# screen_name – Specifies the screen name of the user. Helpful for disambiguating when a valid screen name is also a user ID.
# since_id – Returns only statuses with an ID greater than (that is, more recent than) the specified ID.
# max_id – Returns only statuses with an ID less than (that is, older than) or equal to the specified ID.
# count – Specifies the number of statuses to retrieve.
# page – Specifies the page of results to retrieve. Note: there are pagination limits.

In [10]:
import tweepy
from API_Config_Keys import *

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(Twitter_Consumer_API, Twitter_Consumer_API_Secret)
auth.set_access_token(Twitter_Access_Token, Twitter_Access_Token_Secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [11]:
target_user = "MarsWxReport"
tweet = api.user_timeline(target_user, count =1)

In [12]:
marsweather = tweet[0]['text']
print(marsweather)

Sol 2209 (2018-10-23), high -18C/0F, low -73C/-99F, pressure at 8.79 hPa, daylight 06:09-18:26


# BeautifulSoup append to Dictionary

In [13]:
import time 
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

hemisphere_image_urls = []
hem_image_soup = soup.find("div", class_ = "result-list" )
hemispheres = hem_image_soup.find_all("div", class_="item")

In [14]:
print(len(hemispheres))
print(hemispheres[0])

4
<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>


In [15]:
for x in hemispheres:
    # retrieve title, replace the word 'Enhanced'
    title = x.find("h3").text
    title = title.replace("Enhanced","")
    # retrieve image endpoint
    endpoint_image_link = x.find("a")['href']
    image_link = "https://astrogeology.usgs.gov/" + endpoint_image_link
    # initial BS4
    browser.visit(image_link)
    html = browser.html
    soup = BeautifulSoup(html,"html.parser")
    # find the download links
    soup_downloads = soup.find("div",class_="downloads")
    image_url = soup_downloads.find("a")['href']
    # append titles and links to dictionary
    hemisphere_image_urls.append({"Title":title,"Image":image_url})

In [16]:
hemisphere_image_urls

[{'Title': 'Cerberus Hemisphere ',
  'Image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Schiaparelli Hemisphere ',
  'Image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Title': 'Syrtis Major Hemisphere ',
  'Image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title': 'Valles Marineris Hemisphere ',
  'Image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]